In [9]:
import os
import glob
import shutil
import json
import numpy as np
from PIL import Image
from tqdm.auto import tqdm as tqdm
import numpy as np
import pandas as pd
from itertools import product
import re
np.random.seed(42)

In [27]:
method = 'GAN/GAN2x'
root_dir = os.path.join('..','data', 'data_aug')


data_dir = os.path.join(root_dir, method)
annot_path = os.path.join(root_dir, method, 'labels')

out_dir = data_dir
images_path = os.path.join(root_dir, method, 'images')

json_name = 'coco_format_%s_%s.json'

In [28]:
images = glob.glob(images_path+'/**/*.png', recursive=True)
images

['../data/data_aug/GAN/GAN2x/images/20181022_rectified_DJI_0025/frame_0432.png',
 '../data/data_aug/GAN/GAN2x/images/20181022_rectified_DJI_0025/frame_0576.png',
 '../data/data_aug/GAN/GAN2x/images/20181022_rectified_DJI_0025/frame_0600.png',
 '../data/data_aug/GAN/GAN2x/images/20181022_rectified_DJI_0025/frame_0624.png',
 '../data/data_aug/GAN/GAN2x/images/20181022_rectified_DJI_0025/frame_0648.png',
 '../data/data_aug/GAN/GAN2x/images/20181022_rectified_DJI_0025/frame_0672.png',
 '../data/data_aug/GAN/GAN2x/images/20181022_rectified_DJI_0025/frame_0696.png',
 '../data/data_aug/GAN/GAN2x/images/20181022_rectified_DJI_0025/frame_0744.png',
 '../data/data_aug/GAN/GAN2x/images/20181022_rectified_DJI_0025/frame_0768.png',
 '../data/data_aug/GAN/GAN2x/images/20181022_rectified_DJI_0025/frame_0912.png',
 '../data/data_aug/GAN/GAN2x/images/20181022_rectified_DJI_0025/frame_1488.png',
 '../data/data_aug/GAN/GAN2x/images/20181022_rectified_DJI_0025/frame_1512.png',
 '../data/data_aug/GAN/GAN2x

In [29]:
root = images[0].split(method)[0]
root

'../data/data_aug/'

In [30]:
root = os.path.join(root, method)
root

'../data/data_aug/GAN/GAN2x'

In [31]:
images[0][-14:]

'frame_0432.png'

In [32]:
os.path.join(root, video)

'../data/data_aug/GAN/GAN2x/20191031_rectified_DJI_0017'

In [14]:
# separating images by video folders
r = re.compile(r'\d{8}_rectified_DJI_\d{4}')

for image_path in tqdm(images):
    root = image_path.split(method)[0]
    root = os.path.join(root, method)
        
    video_name = r.findall(image_path)[0]
    frame_name = image_path[-14:]
    
    os.makedirs(os.path.join(images_path, video_name),  exist_ok=True)    
    shutil.move(image_path, os.path.join(images_path, video_name, frame_name))    
    
    
    os.makedirs(os.path.join(images_path.replace('images', 'labels'), video_name),  exist_ok=True) 
    label_path = image_path.replace('images', 'labels').replace('.png', '.txt')    
    shutil.move(label_path, os.path.join(images_path.replace('images', 'labels'), video_name, frame_name.replace('.png', '.txt')))    
    

In [33]:
def get_img_size(datapath):
    img_pil = Image.open(datapath)
    width, height = img_pil.size
    return width, height


def get_groundtruth(img_path, annotation_folder):
    annot_path = None
    if annotation_folder is not None:
        # look for annotation file
        annot_path = _find_annot_file(img_path, annotation_folder)

    if annot_path is not None:
        frame_number = _get_frame_number(img_path)
        annotation = AnnotationImage(frame_number, annot_path)
        boxes, labels = annotation.get_bboxes_labels()

        return boxes, labels

    return [], []

In [34]:
k = 0 #fold
file_set = '../data/v1/train_sets_kfold_v1.0.xls'
df_sets = pd.read_excel(file_set, sheet_name=f'train_sets_k{k}')
df_sets

,Video,seq,bottle,bucket,pool,puddle,tire,water_tank,local,altitude,train,val,test
0,20180905_rectified_DJI_0010,video01,NaN,NaN,NaN,NaN,NaN,NaN,Campo Gremio,10,NaN,NaN,1.0
1,20180919_rectified_DJI_0019,video02,NaN,NaN,NaN,NaN,NaN,NaN,Atras Bloco I,15,NaN,NaN,1.0
2,20180919_rectified_DJI_0021,video03,NaN,NaN,NaN,NaN,NaN,NaN,Acima bloco I (video longo),40,1.0,NaN,NaN
3,20181022_rectified_DJI_0025,video04,NaN,NaN,NaN,NaN,NaN,NaN,aeroclube,15,NaN,1.0,NaN
4,20181022_rectified_DJI_0031,video05,NaN,NaN,NaN,NaN,NaN,NaN,Atras FAU (wild),10,NaN,NaN,1.0
5,20181029_rectified_DJI_0034,video06,NaN,NaN,NaN,NaN,NaN,NaN,CCMN,15,1.0,NaN,NaN
6,20181029_rectified_DJI_0038,video07,NaN,NaN,NaN,NaN,NaN,NaN,frente FAU,20,1.0,NaN,NaN
7,20181112_rectified_DJI_0041,video08,NaN,NaN,NaN,NaN,NaN,NaN,Terreno Atras Bloco I,10,NaN,1.0,NaN
8,20181112_rectified_DJI_0043,video09,NaN,NaN,NaN,NaN,NaN,NaN,Acima bloco I,40,NaN,NaN,1.0
9,20190601_rectified_DJI_0003,video10,NaN,NaN,NaN,NaN,NaN,NaN,Tubiacanga,40,1.0,NaN,NaN


In [35]:
 for (dirpath, dirnames, filenames) in tqdm(os.walk(os.path.join(data_dir))):
        print(dirpath)
        print(dirnames)
        print(filenames)
        
        print('***')

../data/data_aug/GAN/GAN2x
['images', 'labels']
['coco_format_GAN2x_train1.json', 'coco_format_GAN2x_train2.json', 'coco_format_GAN2x_train3.json', 'coco_format_GAN2x_train4.json', 'coco_format_GAN2x_train5.json', 'coco_format_GAN2x_train6.json', 'coco_format_GAN2x_train7.json', 'coco_format_GAN2x_train+val.json', 'coco_format_GAN2x_train0.json']
***
../data/data_aug/GAN/GAN2x/images
['20181022_rectified_DJI_0025', '20181029_rectified_DJI_0034', '20181029_rectified_DJI_0038', '20181112_rectified_DJI_0041', '20190601_rectified_DJI_0003', '20190601_rectified_DJI_0005', '20191031_rectified_DJI_0017']
[]
***
../data/data_aug/GAN/GAN2x/images/20181022_rectified_DJI_0025
[]
['frame_0432.png', 'frame_0576.png', 'frame_0600.png', 'frame_0624.png', 'frame_0648.png', 'frame_0672.png', 'frame_0696.png', 'frame_0744.png', 'frame_0768.png', 'frame_0912.png', 'frame_1488.png', 'frame_1512.png', 'frame_1608.png', 'frame_1632.png']
***
../data/data_aug/GAN/GAN2x/images/20181029_rectified_DJI_0034
[]
[

In [37]:
import re

include_reals = True # please, manually check if it is already included in the file

mask = df_sets['test'] == True
videos_train_val = df_sets[~mask]['Video'].tolist()

sets = [f'train{n}' for n in np.arange(len(videos_train_val))]
sets += ['train+val']


category_dict = {
    #'__background__': 0,
    'tire': 0,
    }

for subset in tqdm(sets):
    print(f'Starting {method, subset}')
    
    if 'train+val' in subset:
        videos_set = videos_train_val
        
        
    elif 'train' in subset:
        idx_val = int(subset[-1])
        videos_set = [x for i, x in enumerate(videos_train_val) if i != idx_val]        
        
    ann_dict = {}
    images = []
    annotations = []
        
    img_id = 0
    ann_id = 0
    cat_id = 1
    
#     filenames = glob.glob(os.path.join(data_dir, data_set,'*.png'))
    
    for (dirpath, dirnames, filenames) in tqdm(os.walk(os.path.join(data_dir))):
        
        if len(filenames) == 0:
            continue
    
        video_name = dirpath.split('/')[-1]                
        if video_name not in videos_set:            
            continue
        
        print(video_name)
        for filename in tqdm(filenames):               
            if filename.lower().endswith('.png'):                          
                image = {}
                
                img_name_short = os.path.join('images', video_name, filename)
                img_name = os.path.join(dirpath, filename)
                
                    
                width, height = get_img_size(img_name)
                
                image['file_name'] = img_name_short
                image['width'] = width
                image['height'] = height
                image['id'] = img_id
                                
                #read .txt in images folder
                if include_reals:                    
                    anno_filepath = os.path.join(root_dir, "GAN", "labels_reals", 
                                                 f"{video_name.replace('rectfied', 'rectified')}_{filename.replace('.png','.txt')}")
                    
                    if os.path.isfile(anno_filepath):
                        num_real_in_img = 0
                        
                        with open(anno_filepath, 'r') as marks:
                            for mark in marks.readlines():  
                                rect = mark.replace('\n','').split('\t') 
                                
                                
                                
                                if len(rect) <=1:                                    
                                    continue
                                    
                                objid,rcx,rcy,rw,rh = map(float,rect)
                                x0 = int((rcx-rw/2)*width) #hardcoded pq veio assim da annot
                                y0 = int((rcy-rh/2)*height)
                                w = int(rw*width)
                                h = int(rh*height)

                                ann = {}
                                ann['id'] = ann_id
                                ann['image_id'] = img_id
                                ann['category_id'] = 0
                                ann['iscrowd'] = 0
                                ann['area'] = w*h
                                ann['bbox'] = (x0,y0,w,h)

                                annotations.append(ann)
                                ann_id += 1    
                                num_real_in_img += 1      
                                
                #read .txt in labels folder                                
                anno_filepath = os.path.join(data_dir, 'labels', video_name, filename.replace('.png','.txt'))                

                with open(anno_filepath, 'r') as marks:

                    for mark in marks.readlines(): 
                        print(anno_filepath)
                        print(mark)
                        
                        rect = mark.replace('\n','')
#                       
                        if '\t' in rect:
                            rect = rect.split('\t')                                                                  
                        if ' ' in rect:
                            rect = rect.split(' ')  
                        
                        print(rect)
                        print('\n')

                        if len(rect) <=1:            
                            continue

                        objid,rcx,rcy,rw,rh = map(float,rect)
                    
                        
                        x0 = int((rcx-rw/2)*width) 
                        y0 = int((rcy-rh/2)*height)
                        w = int(rw*width)
                        h = int(rh*height)                            

                        ann = {}
                        ann['id'] = ann_id
                        ann['image_id'] = img_id
                        ann['category_id'] = 0
                        ann['iscrowd'] = 0
                        ann['area'] = w*h
                        ann['bbox'] = (x0,y0,w,h)

                        annotations.append(ann)
                        ann_id += 1
                        
                images.append(image)
                img_id += 1

            
    ann_dict['images'] = images
    categories = [{"id": category_dict[name], "name": name} for name in category_dict]
    ann_dict['categories'] = categories
    ann_dict['annotations'] = annotations
    print("Num categories: %s" % len(categories))
    print("Num images: %s" % len(images))
    print("Num annotations: %s" % len(annotations))
    
    
    with open(os.path.join(out_dir, json_name % (method.replace('/GAN2x', ''), subset)), 'w') as outfile:
        outfile.write(json.dumps(ann_dict))

Starting ('GAN/GAN2x', 'train0')


20181022_rectified_DJI_0025


../data/data_aug/GAN/GAN2x/labels/20181022_rectified_DJI_0025/frame_0432.txt
0	0.3280590717299578	0.724375	0.027426160337552744	0.04375

['0', '0.3280590717299578', '0.724375', '0.027426160337552744', '0.04375']


../data/data_aug/GAN/GAN2x/labels/20181022_rectified_DJI_0025/frame_0432.txt
0	0.46659634317862164	0.321875	0.027426160337552744	0.04375

['0', '0.46659634317862164', '0.321875', '0.027426160337552744', '0.04375']


../data/data_aug/GAN/GAN2x/labels/20181022_rectified_DJI_0025/frame_0576.txt
0	0.20991561181434598	0.96	0.027426160337552744	0.0425

['0', '0.20991561181434598', '0.96', '0.027426160337552744', '0.0425']


../data/data_aug/GAN/GAN2x/labels/20181022_rectified_DJI_0025/frame_0576.txt
0	0.3062587904360056	0.2225	0.027426160337552744	0.04

['0', '0.3062587904360056', '0.2225', '0.027426160337552744', '0.04']


../data/data_aug/GAN/GAN2x/labels/20181022_rectified_DJI_0025/frame_0600.txt
0	0.8424753867791842	0.773125	0.02812939521800281	0.04875

['0', '0.842475386779184

../data/data_aug/GAN/GAN2x/labels/20181029_rectified_DJI_0034/frame_0000.txt
0	0.5443037974683544	0.240625	0.02531645569620253	0.04375

['0', '0.5443037974683544', '0.240625', '0.02531645569620253', '0.04375']


../data/data_aug/GAN/GAN2x/labels/20181029_rectified_DJI_0034/frame_0000.txt
0	0.15154711673699015	0.84375	0.026019690576652602	0.045

['0', '0.15154711673699015', '0.84375', '0.026019690576652602', '0.045']


../data/data_aug/GAN/GAN2x/labels/20181029_rectified_DJI_0034/frame_0000.txt
0	0.18354430379746836	0.754375	0.02531645569620253	0.03625

['0', '0.18354430379746836', '0.754375', '0.02531645569620253', '0.03625']


../data/data_aug/GAN/GAN2x/labels/20181029_rectified_DJI_0034/frame_0024.txt
0	0.9511251758087201	0.123125	0.02461322081575246	0.03875

['0', '0.9511251758087201', '0.123125', '0.02461322081575246', '0.03875']


../data/data_aug/GAN/GAN2x/labels/20181029_rectified_DJI_0034/frame_0024.txt
0	0.9349507735583685	0.366875	0.02461322081575246	0.04375

['0', '0.9349507

../data/data_aug/GAN/GAN2x/labels/20181029_rectified_DJI_0038/frame_0336.txt
0	0.6244725738396625	0.296875	0.02109704641350211	0.03375

['0', '0.6244725738396625', '0.296875', '0.02109704641350211', '0.03375']


../data/data_aug/GAN/GAN2x/labels/20181029_rectified_DJI_0038/frame_0336.txt
0	0.1979606188466948	0.271875	0.020393811533052038	0.03375

['0', '0.1979606188466948', '0.271875', '0.020393811533052038', '0.03375']


../data/data_aug/GAN/GAN2x/labels/20181029_rectified_DJI_0038/frame_0360.txt
0	0.8466947960618847	0.2725	0.02109704641350211	0.035

['0', '0.8466947960618847', '0.2725', '0.02109704641350211', '0.035']


../data/data_aug/GAN/GAN2x/labels/20181029_rectified_DJI_0038/frame_0360.txt
0	0.8122362869198312	0.22625	0.02109704641350211	0.035

['0', '0.8122362869198312', '0.22625', '0.02109704641350211', '0.035']


../data/data_aug/GAN/GAN2x/labels/20181029_rectified_DJI_0038/frame_0360.txt
0	0.11392405063291139	0.86125	0.02109704641350211	0.0375

['0', '0.11392405063291139', 

...


20181112_rectified_DJI_0041



20190601_rectified_DJI_0003



20190601_rectified_DJI_0005



20191031_rectified_DJI_0017




Num categories: 1
Num images: 171
Num annotations: 808



In [21]:
out_dir

'../data/data_aug/GAN/set_0'